In [1]:
cd ..

/home/luco/python/ml-imdb


# Movie Rating Prediction from Graph Statistics

In [2]:
import json
import numpy as np

from commonstuff import *

In [3]:
from sklearn.preprocessing import normalize
from sklearn.svm import LinearSVR
from sklearn.metrics import mean_squared_error, mean_absolute_error

from keras.models import Sequential
from keras.layers import Input, Dense, Dropout
from keras.activations import relu
from keras.optimizers import SGD

Using TensorFlow backend.


In [13]:
def get_data(datafn, testfn, normalized=True):

    with open(datafn) as file:
        data = json.load(file)
    with open(testfn) as file:
        test = json.load(file)

    x_train = np.array(map(lambda x: np.array(x['attributes'], dtype = 'f'), data['data']))
    x_test = np.array(map(lambda x: np.array(x['attributes'], dtype = 'f'), test['data']))

    if normalized:
        # normalizing by features
        # e.g. betweeness as a very high number corrupts models
        normalize(x_train, axis=0, copy=False)
        normalize(x_test, axis=0, copy=False)
    
    y_train = np.array(map(lambda x: int(round(x['target'])), data['data']))
    y_train = y_train.reshape(y_train.size,1).ravel()    
    y_test = np.array(map(lambda x: int(round(x['target'])), test['data']))
    y_test = y_test.reshape(y_test.size,1).ravel()
    
    return x_train, y_train, x_test, y_test

In [14]:
train_fn = "./data_1980_2000.json"
test_fn = "./data_2000_2005.json"

In [15]:
x_train, y_train, x_test, y_test = get_data(train_fn, test_fn)

In [16]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((1713, 34), (1713,), (1280, 34), (1280,))

## Support Vector Regression

In [17]:
model = LinearSVR()
model.fit(x_train, y_train)

LinearSVR(C=1.0, dual=True, epsilon=0.0, fit_intercept=True,
     intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=1000,
     random_state=None, tol=0.0001, verbose=0)

In [18]:
mean_absolute_error(model.predict(x_test), y_test)

1.0828771047420482

## Feedforward Neural Net

In [19]:
model = Sequential()

input_shape = (x_train.shape[1], )

model.add(Dense(160, activation=relu, input_shape=input_shape))
model.add(Dropout(0.4))
model.add(Dense(160, activation=relu))
model.add(Dropout(0.4))
model.add(Dense(1, activation=relu))

In [20]:
model.compile(loss="mse", metrics=["mae"], optimizer="adadelta")

In [21]:
model.fit(x_train, y_train,
          validation_data=(x_test, y_test),
          epochs=100, batch_size=500, verbose=1)

Train on 1713 samples, validate on 1280 samples
Epoch 1/100
1713/1713 [==============================] - 0s - loss: 40.0895 - mean_absolute_error: 6.1769 - val_loss: 41.8153 - val_mean_absolute_error: 6.3029
Epoch 2/100
1713/1713 [==============================] - 0s - loss: 39.4193 - mean_absolute_error: 6.1221 - val_loss: 40.7537 - val_mean_absolute_error: 6.2182
Epoch 3/100
1713/1713 [==============================] - 0s - loss: 38.2033 - mean_absolute_error: 6.0217 - val_loss: 38.5046 - val_mean_absolute_error: 6.0350
Epoch 4/100
1713/1713 [==============================] - 0s - loss: 35.2952 - mean_absolute_error: 5.7741 - val_loss: 32.8627 - val_mean_absolute_error: 5.5481
Epoch 5/100
1713/1713 [==============================] - 0s - loss: 27.9633 - mean_absolute_error: 5.0895 - val_loss: 19.8079 - val_mean_absolute_error: 4.2155
Epoch 6/100
1713/1713 [==============================] - 0s - loss: 13.8012 - mean_absolute_error: 3.3989 - val_loss: 4.4632 - val_mean_absolute_error: 

1713/1713 [==============================] - 0s - loss: 2.0948 - mean_absolute_error: 1.1514 - val_loss: 1.8113 - val_mean_absolute_error: 1.0626
Epoch 53/100
1713/1713 [==============================] - 0s - loss: 2.0949 - mean_absolute_error: 1.1450 - val_loss: 1.8046 - val_mean_absolute_error: 1.0561
Epoch 54/100
1713/1713 [==============================] - 0s - loss: 2.1607 - mean_absolute_error: 1.1681 - val_loss: 1.8483 - val_mean_absolute_error: 1.0829
Epoch 55/100
1713/1713 [==============================] - 0s - loss: 2.0859 - mean_absolute_error: 1.1423 - val_loss: 1.8407 - val_mean_absolute_error: 1.0796
Epoch 56/100
1713/1713 [==============================] - 0s - loss: 2.0399 - mean_absolute_error: 1.1411 - val_loss: 1.8011 - val_mean_absolute_error: 1.0502
Epoch 57/100
1713/1713 [==============================] - 0s - loss: 2.1549 - mean_absolute_error: 1.1621 - val_loss: 1.8393 - val_mean_absolute_error: 1.0791
Epoch 58/100
1713/1713 [==============================] - 0